In [44]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import sys

In [45]:
filename = "alice_in_wonderland.txt"  # Update with your dataset
raw_text = open(filename, "r", encoding="utf-8").read().lower()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [46]:
# Create training patterns
seq_len = 100
dataX = []
dataY = []
for i in range(0, len(raw_text) - seq_len, 1):
    seq_in = raw_text[i:i + seq_len]
    seq_out = raw_text[i + seq_len]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
n_vocab = len(chars)

X = numpy.reshape(dataX, (n_patterns, seq_len, 1))
X = X / float(n_vocab)
y = to_categorical(dataY)

In [47]:
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))

In [48]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001))

In [49]:
filepath = "weights-improvement-{epoch:01d}-{loss:.4f}.hdf5"  # Store path and type
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1160/1160 [==============================] - ETA: 0s - loss: 3.0552
Epoch 1: loss improved from inf to 3.05517, saving model to weights-improvement-1-3.0552.hdf5
1160/1160 [==============================] - 1028s 883ms/step - loss: 3.0552
Epoch 2/20
1061/1160 [==========================>...] - ETA: 1:28 - loss: 3.0106

In [ ]:
f_name = "weights-improvement-14-2.0773.hdf5"
model.load_weights(f_name)

In [ ]:
def generate_response(prompt, model, char_to_int, int_to_char, seq_len=100, n_vocab=0):
    # Preprocess input
    prompt = prompt.lower()
    input_pattern = [char_to_int[char] for char in prompt if char in char_to_int]
    
    # Pad or trim input to match sequence length
    while len(input_pattern) < seq_len:
        input_pattern.insert(0, 0)  # Pad with zero
    input_pattern = input_pattern[-seq_len:]
    # Generate response
    generated_text = ""
    for _ in range(200):  # Limit response length
        x = numpy.reshape(input_pattern, (1, seq_len, 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        generated_text += result
        
        # Update input pattern
        input_pattern.append(index)
        input_pattern = input_pattern[1:]
    
    return generated_text.strip()

In [ ]:
# Chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = generate_response(user_input, model, char_to_int, int_to_char, seq_len, n_vocab)
    print(f"Chatbot: {response}")